The phrase "Saving a TensorFlow model" typically means one of two things:
1. Checkpoints, OR
2. SavedModel.

Checkpoints capture the exact value of all parameters (`tf.Variable` objects) used by a model. Checkpoints do not contain any description of the computation defined by the model and thus are typically only useful when source code that will use the saved parameter values is abailable.

The SavedModel format on the other hand includes a serialized description of the computation defined by the model in addition to the parameter values (checkpoint). Models in this format are independent of the source code that created the model. They are thus suitable for deployment via TensorFlow Serving, TensorFlow Lite, Tensorflow.js, or programs in other programming languages (the C, C++, Java, Go, Rust, C# etc. Tensorflow APIs).

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

### Saving from `tf.keras` training APIs
See the `tf.keras` guide on saving and restoring.

`tf.keras.Model.save_weights` saves a Tensorflow checkpoint.

In [0]:
net.save_weights('easy_checkpoint')

### Writing checkpoints
The persistent state of a TensorFlow model is stored in `tf.Variable` objects. These can be constructed directly, but are often created through high-level APIs like `tf.keras.layers` or tf.keras.Model.

The easiest way to manage variables is by attaching them to Python objects, then referencing those objects.

Subclasses of `tf.train.Checkpoint`, tf.keras.layers.Layer`, and `tf.keras.Model` automatically track variables assigned to their attributes. The following example constructs a simple linear model, then writes checkpoints which contain values for all of the model's variables.

You can easily save a model-checkpoint with `Model.save_weights`

### Manual checkpointing
### Setup
To help demonstrate all the features of `tf.train.Checkpoint` define a toy dataset and optimization step:

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
      dict(x=inputs, y=labels)).repeat(10).batch(2)

In [0]:
tf.range(10.)

In [0]:
inputs = tf.range(10.)[:, None] # None is newaxis.
inputs

In [0]:
 inputs * 5. + tf.range(5.)[None, :]
 # inputs is shape (10, 1)
 # tf.range(5.)[None, :] is shape (1, 5)
# broadcasting...

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

### Create the checkpoint objects
To manually make a checkpoint you will need a `tf.train.Checkpoint` object. Where the objects you want to checkpoint are set as attributes on the object.

A `tf.train.CheckpointManager` can also be helpful for managing multiple checkpoints.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, 
                           minho=tf.Variable(2))
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

### Train and checkpoint the model
The following training loop creates an instance of the model and of an optimizer, then gathers them into a `tf.train.Checkpoint` object. It calls the training step in a loop on each batch of data, and periodically writes checkpoints to disk.

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print('Restored from {}'.format(manager.latest_checkpoint))
  else:
    print('Initializing from scratch.')
  
  for example in toy_dataset():
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1) # 아 이렇게 attribute 접근이 가능하구나...
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print('Saved checkpoint for step {}: {}'.format(int(ckpt.step), save_path))
      print('loss {:1.2f}'.format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

In [0]:
ckpt.minho

### Restore and continue training
After the first you can pass a new model and manager, but pickup training exactly where you left off:

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, minho=tf.Variable(3))
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

In [0]:
ckpt.minho # Is 2!

The `tf.train.CheckpointManager` object deletes old checkpoints. Above it's configured to keep only the three most recent checkpoints.

In [0]:
print(manager.checkpoints)

These path, e.g. `tf./tf_ckpts/ckpt-10`, are not files on disk. Instead they are prefixes for an `index` file and one or more data files which contain the variable values. These prefixes are grouped together in a single `checkpoint` file (`./tf_ckpts/checkpoint`) where the `CheckpointManager` saves its state.

In [0]:
!ls ./tf_ckpts/

In [0]:
manager.latest_checkpoint

## Loading mechanics
TensorFlow mathes variables to checkpointed values by traversing a directed graph with named edges, starting from the object being loaded. Edge names typically come from attribute names in objects, for example the "`ll`" in `self.l` = tf.keras.layers.Dense(5)`. `tf.train.Checkpoint` uses its keyword argument names, as in the "`step`" in `tf.train.Checkpoint(step=...).`



Calling `restore()` on a `tf.train.Checkpoint` object queues the requested restorations, restoring variable values as soon as there's a matching path from the `Checkpoint` object. For example we can load just the kernel from the model we defined above by reconstructing one path to it through the network and the layer.

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy()) # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy()) # We get restored value now